In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
def process_data(file_name):
    words=[]
    with open(file_name) as f:
        file_name_data = f.read()
    file_name_data=file_name_data.lower()
    words = re.findall('\w+',file_name_data)
    return words

In [3]:
def get_count(data):
    word_count_dict = {}
    word_count_dict = Counter(data)        
    return word_count_dict

In [4]:
def get_probs(word_count_dict):
    probs = {}
    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    return probs

In [5]:
data = process_data('/home/sagar24/nlp codes/shakespeare.txt')
vocab = set(data)  
word_count_dict = get_count(data)
print(f"# Unique words in Vocab : {len(vocab)} ")
probs = get_probs(word_count_dict)

# Unique words in Vocab : 6116 


In [6]:
def delete_letter(word):
    delete_l = []
    split_l = []
    for c in range(len(word)):
        split_l.append((word[:c],word[c:]))
    for a,b in split_l:
        delete_l.append(a+b[1:])
    #print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")
    return delete_l

def switch_letter(word): 
    switch_l = []
    split_l = []
    len_word=len(word)
    for c in range(len_word):
        split_l.append((word[:c],word[c:]))
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]
    #print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 
    return switch_l

def replace_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    for c in range(len(word)):
        split_l.append((word[0:c],word[c:]))
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)    
    replace_set.remove(word)
    replace_l = sorted(list(replace_set))
    #print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    return replace_l

def insert_letter(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    for c in range(len(word)+1):
        split_l.append((word[0:c],word[c:]))
    insert_l = [ a + l + b for a,b in split_l for l in letters]
    #print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    return insert_l

In [7]:
def edit_one_letter(word, allow_switches = True):
    edit_one_set = set()
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    return edit_one_set

def edit_two_letters(word, allow_switches = True):
    edit_two_set = set()
    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    return edit_two_set

In [8]:
def get_corrections(word, probs, vocab, n=2):
    suggestions = []
    n_best = []
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]
    print("entered word = ", word, "\nsuggestions = ", suggestions)
    return n_best

In [9]:
my_word = 'shakespare' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

entered word =  shakespare 
suggestions =  ['shakespeare']
word 0: shakespeare, probability 0.000168


In [10]:
my_word = 'cerriage' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

entered word =  cerriage 
suggestions =  ['carriage']
word 0: carriage, probability 0.000019


In [11]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    m = len(source) 
    n = len(target) 
    D = np.zeros((m+1, n+1), dtype=int) 
    for row in range(1,m+1): 
        D[row,0] = D[row-1,0] + del_cost
        
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + ins_cost
        
    for row in range(1,m+1): 
        for col in range(1,n+1):
            r_cost = rep_cost
            if source[row-1] == target[col-1]:
                r_cost = 0    
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
    med = D[m,n]
    return D, med

In [12]:
w1="favore"
targets = edit_one_letter(w1,allow_switches = False)
for t in targets:
    _, min_edits = min_edit_distance(w1, t,1,1,1)  
    if min_edits != 1: 
        print(w1, t, min_edits)

In [13]:
w1="favore"
targets = edit_two_letters(w1,allow_switches = False)  
for t in targets:
    _, min_edits = min_edit_distance(w1, t,1,1,1)  
    if min_edits != 2 and min_edits != 1: 
        print(w1, t, min_edits)
    

favore favore 0
